In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time


Importe les bibliothèques nécessaires pour les calculs numériques (numpy), la visualisation (matplotlib, mplot3d pour les graphiques 3D).

In [ ]:
class TSNE:
    def __init__(self, n_components=2, perplexity=30.0, 
                 early_exaggeration=12.0, learning_rate=200.0, 
                 n_iter=1000, min_grad_norm=1e-7, 
                 random_state=None, verbose=0):
         """
        Implémentation from scratch de t-SNE.
        
        Paramètres:
        -----------
        n_components : int, (défaut: 2)
            Dimension de l'espace embarqué
            
        perplexity : float, (défaut: 30)
            Contrôle le nombre de voisins locaux considérés
            
        early_exaggeration : float, (défaut: 12.0)
            Facteur d'exagération initial pour séparer les clusters
            
        learning_rate : float, (défaut: 200.0)
            Taux d'apprentissage pour la descente de gradient
            
        n_iter : int, (défaut: 1000)
            Nombre maximal d'itérations
            
        min_grad_norm : float, (défaut: 1e-7)
            Seuil minimal de la norme du gradient pour continuer
            
        random_state : int ou None, (défaut: None)
            Graine pour le générateur aléatoire
            
        verbose : int, (défaut: 0)
            Niveau de verbosité (0: silencieux, 1: progress, 2: détaillé)
        """
        self.n_components = n_components
        self.perplexity = perplexity
        self.early_exaggeration = early_exaggeration
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.min_grad_norm = min_grad_norm
        self.random_state = random_state
        self.verbose = verbose
        
        # Résultats
        self.embedding_ = None
        self.kl_divergence_ = None
        self.n_iter_ = None
    
        # Variables internes
        if random_state is not None:
            np.random.seed(random_state)
            
        '''Si une graine aléatoire est fournie, on l'utilise pour 
        initialiser le générateur de nombres aléatoires de NumPy 
        (pour reproductibilité).'''

Paramètres:
-----------

- `n_components`: Paramètre qui définit dans quelle dimension l'espace sera réduit (généralement 2 ou 3 pour la visualisation).


- `perplexity`: Paramètre qui contrôle le nombre effectif de voisins considérés pour chaque point. Une valeur typique est entre 5 et 50.


- `early_exaggeration`: Facteur qui exagère les distances au début de l'optimisation pour mieux séparer les clusters.


- `learning_rate`: Contrôle la taille des pas dans la descente de gradient. Trop élevé peut causer une divergence, trop bas une convergence lente.


- `n_iter`: Nombre maximum d'itérations pour l'optimisation.


- `min_grad_norm`: Seuil en dessous duquel l'optimisation s'arrête (considérée comme convergée).


- `random_state`: Graine aléatoire pour la reproductibilité des résultats.


- `verbose`: Contrôle la quantité d'informations affichées pendant l'exécution.

- Initialisation des variables qui stockeront les résultats:
  - `embedding_`: La projection des données en dimension réduite
  - `kl_divergence_`: La divergence KL finale (mesure de qualité)
  - `n_iter_`: Le nombre d'itérations effectuées


In [ ]:
def _euclidean_distance(self, X):
    """Calcule la matrice des distances euclidiennes carrées entre tous les points."""
    sum_X = np.sum(np.square(X), axis=1)
    distances = np.add(-2 * np.dot(X, X.T), sum_X).T + sum_X
    np.fill_diagonal(distances, 0.0)
    return distances

 Méthode qui calcule efficacement les distances euclidiennes carrées entre toutes les paires de points:
  1. `sum_X = np.sum(np.square(X), axis=1)` - Calcule la somme des carrés pour chaque point
  2. `distances = np.add(-2 * np.dot(X, X.T), sum_X).T + sum_X` - Formule mathématique optimisée pour calculer ||x-y||² = ||x||² + ||y||² - 2x·y
  3. `np.fill_diagonal(distances, 0.0)` - Met les distances à soi-même à 0
  4. Retourne la matrice de distances

In [ ]:
 def _binary_search_perplexity(self, distances, perplexity, tol=1e-5, max_iter=50):
        """Trouve les sigma appropriés pour obtenir la perplexité souhaitée."""
        n_samples = distances.shape[0]
        P = np.zeros((n_samples, n_samples))
        beta = np.ones((n_samples, 1))
        log_perplexity = np.log(perplexity)

 Méthode qui trouve les paramètres sigma (via beta=1/(2σ²)) pour obtenir la perplexité désirée:
  1. Initialise la matrice de probabilités P à zéro
  2. Initialise beta (inverse de la variance) à 1 pour tous les points
  3. Calcule le log de la perplexité cible

In [ ]:
# On ignore la diagonale (distance à soi-même = 0)
        for i in range(n_samples):
            beta_min = -np.inf
            beta_max = np.inf
            dist_i = distances[i, np.concatenate((np.r_[0:i], np.r_[i+1:n_samples]))]
            

Pour chaque point i:
  1. Initialise les bornes de recherche binaire
  2. Extrait les distances de i à tous les autres points (en excluant i lui-même)

In [ ]:

            for _ in range(max_iter):
                # Calcul des probabilités conditionnelles
                P_i = np.exp(-dist_i * beta[i])
                sum_Pi = np.sum(P_i)
                
                if sum_Pi == 0:
                    sum_Pi = 1e-8
                
                # Calcul de l'entropie
                H = np.log(sum_Pi) + beta[i] * np.sum(dist_i * P_i) / sum_Pi
                P_i = P_i / sum_Pi
                
                # Ajustement de beta (précision binaire)
                H_diff = H - log_perplexity
                if np.abs(H_diff) < tol:
                    break

Recherche binaire pour trouver le bon beta:
  1. Calcule les probabilités conditionnelles P(j|i)
  2. Gère le cas où la somme est nulle pour éviter la division par zéro
Calcule l'entropie H (qui doit correspondre à log(perplexity))
- Normalise les probabilités
- Calcule l'écart à la cible et arrête si suffisamment proche

In [ ]:

                if H_diff > 0:
                    beta_min = beta[i].copy()
                    if beta_max == np.inf:
                        beta[i] *= 2.0
                    else:
                        beta[i] = (beta[i] + beta_max) / 2.0
                else:
                    beta_max = beta[i].copy()
                    if beta_min == -np.inf:
                        beta[i] /= 2.0
                    else:
                        beta[i] = (beta[i] + beta_min) / 2.0
            
            # Remplir la matrice P
            P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n_samples]))] = P_i
        
        return P

Ajuste beta selon si l'entropie est trop haute ou trop basse:
  - Si H > log(perplexity): besoin de diminuer beta (augmenter sigma)
  - Si H < log(perplexity): besoin d'augmenter beta (diminuer sigma)

  Stocke les probabilités trouvées dans la matrice P
- Retourne la matrice de probabilités conditionnelles P(j|i)

Cette implémentation montre les parties clés de t-SNE: calcul des distances, recherche des bonnes variances pour obtenir la perplexité souhaitée, et calcul des probabilités dans l'espace de départ.


In [ ]:
def _compute_joint_probabilities(self, X, perplexity):
        """Calcule les probabilités jointes p_ij."""
        # Calcul des distances euclidiennes carrées
        distances = self._euclidean_distance(X)
        
        # Calcul des probabilités conditionnelles
        P = self._binary_search_perplexity(distances, perplexity)
        
        # Symétrisation et normalisation
        P = (P + P.T) / (2.0 * P.shape[0])
        P = np.maximum(P, 1e-12)
        
        return P

Cette méthode calcule les probabilités jointes `p_ij` qui représentent les similarités entre les points dans l'espace de haute dimension. Voici ce que fait chaque partie:

## 1. Calcul des distances euclidiennes carrées
```python
distances = self._euclidean_distance(X)
```
- **Action**: Calcule la matrice des distances euclidiennes carrées entre toutes les paires de points dans l'espace original
- **Détail technique**: Utilise la formule optimisée `||x-y||² = ||x||² + ||y||² - 2x·y`
- **Résultat**: Matrice carrée de taille (n_samples × n_samples) où chaque élément [i,j] contient la distance entre le point i et j

## 2. Calcul des probabilités conditionnelles
```python
P = self._binary_search_perplexity(distances, perplexity)
```
- **Action**: Trouve les probabilités conditionnelles P(j|i) via recherche binaire
- **Fonctionnement**:
  - Pour chaque point i, trouve un σ (sigma) tel que la distribution de probabilité sur ses voisins ait la perplexité souhaitée
  - La perplexité contrôle le nombre effectif de voisins considérés
- **Résultat**: Matrice P où P[i,j] = probabilité que j soit un voisin de i

## 3. Symétrisation des probabilités
```python
P = (P + P.T) / (2.0 * P.shape[0])
```
- **Problème**: Les P(j|i) ne sont pas symétriques (P(j|i) ≠ P(i|j))
- **Solution**: On crée des probabilités jointes symétriques en faisant:
  - Moyenne entre P(j|i) et P(i|j) via `(P + P.T)`
  - Normalisation par `2*N` pour que la somme totale soit 1
- **But**: Obtenir une mesure de similarité symétrique entre paires de points

## 4. Éviter les valeurs numériques trop petites
```python
P = np.maximum(P, 1e-12)
```
- **Problème**: Certaines probabilités pourraient devenir trop proches de zéro
- **Solution**: On impose une valeur minimale de 10⁻¹²
- **But**: Éviter:
  - Des divisions par zéro
  - Des problèmes avec le calcul du logarithme dans la divergence KL
  - Des instabilités numériques dans l'optimisation

## Résultat final
- Retourne une matrice de probabilités jointes symétrique P où:
  - P[i,j] = probabilité que i et j soient voisins
  - Plus P[i,j] est grand, plus les points sont similaires
  - La somme totale des P[i,j] vaut 1

  ## Importance dans t-SNE
Ces probabilités P représentent la structure de voisinage dans l'espace de haute dimension que t-SNE tentera de préserver dans l'espace de faible dimension. La qualité de ce calcul influence directement la qualité de la visualisation finale.




In [ ]:
  def _compute_low_dimensional_probabilities(self, Y):
        """Calcule les probabilités q_ij en basse dimension."""
        distances = self._euclidean_distance(Y)
        inv_distances = 1.0 / (1.0 + distances)
        np.fill_diagonal(inv_distances, 0.0)
        Q = inv_distances / np.sum(inv_distances)
        Q = np.maximum(Q, 1e-12)
        return Q

  def _compute_low_dimensional_probabilities(self, Y):
- Calcule les probabilités q_ij en basse dimension.
- **Finalité** : Calcule la matrice de similarité `Q` entre les points dans l'espace réduit (2D/3D).


distances = self._euclidean_distance(Y)
- **Effet** : Calcule les distances euclidiennes carrées entre tous les points dans l'espace réduit.
- **Finalité** : Permet de mesurer à quel point les points sont proches dans la visualisation.


    inv_distances = 1.0 / (1.0 + distances)
- **Effet** : Applique une loi de Student (t-distribution) pour convertir les distances en similarités.
- **Finalité** : Les points proches auront une similarité élevée (~1), les points éloignés une similarité faible (~0).


    np.fill_diagonal(inv_distances, 0.0)
- **Effet** : Met à zéro la diagonale (un point n'est pas similaire à lui-même).
- **Finalité** : Évite les auto-similarités qui fausseraient la normalisation.


    Q = inv_distances / np.sum(inv_distances)
- **Effet** : Normalise les similarités pour obtenir une distribution de probabilité.
- **Finalité** : `Q` représente maintenant la probabilité que deux points soient voisins en 2D/3D.


    Q = np.maximum(Q, 1e-12)
- **Effet** : Évite les valeurs trop petites (problèmes numériques).
- **Finalité** : Garantit la stabilité des calculs (évite `log(0)`).


    return Q
- **Finalité** : Retourne la matrice `Q` qui sera utilisée pour calculer le gradient.


In [ ]:
 def _compute_gradient(self, P, Q, Y):
        """Calcule le gradient de la divergence KL par rapport à l'embedding."""
        n = Y.shape[0]
        gradient = np.zeros_like(Y)
        
        # Calcul des termes (p_ij - q_ij) * (1 + ||y_i - y_j||²)^-1
        dist = 1.0 / (1.0 + self._euclidean_distance(Y))
        pq_diff = (P - Q) * dist
        
        # Calcul du gradient
        for i in range(n):
            gradient[i] = 4.0 * np.sum((Y[i] - Y) * pq_diff[:, i][:, np.newaxis], axis=0)
        
        return gradient

def _compute_gradient(self, P, Q, Y):
    """Calcule le gradient de la divergence KL par rapport à l'embedding."""
- **Finalité** : Calcule comment ajuster les positions `Y` pour minimiser l'erreur entre `P` (hautes dimensions) et `Q` (basses dimensions).


    n = Y.shape[0]
- **Effet** : Stocke le nombre de points.


    gradient = np.zeros_like(Y)
- **Effet** : Initialise un gradient de même forme que `Y` (rempli de zéros).
- **Finalité** : Stockera les dérivées de la divergence KL par rapport à chaque point.


    dist = 1.0 / (1.0 + self._euclidean_distance(Y))
- **Effet** : Recalcule les similarités en 2D/3D (comme dans `Q`).
- **Finalité** : Utilisé pour pondérer les différences entre `P` et `Q`.


    pq_diff = (P - Q) * dist
- **Effet** : Calcule `(P - Q) × (similarité en 2D)`.
- **Finalité** : Plus la similarité `Q` est éloignée de `P`, plus le gradient sera fort.


    for i in range(n):
        gradient[i] = 4.0 * np.sum((Y[i] - Y) * pq_diff[:, i][:, np.newaxis], axis=0)
- **Effet** : Pour chaque point, calcule une force d'attraction/répulsion basée sur `(P - Q)`.
- **Finalité** :  
  - Si `P > Q` (trop éloignés en 2D), le point sera attiré.  
  - Si `P < Q` (trop proches en 2D), le point sera repoussé.


    return gradient
- **Finalité** : Retourne le gradient qui sera utilisé pour déplacer les points.


In [ ]:
def _compute_kl_divergence(self, P, Q):
        """Calcule la divergence KL entre P et Q."""
        return np.sum(P * np.log(P / Q))
    

def _compute_kl_divergence(self, P, Q):
    """Calcule la divergence KL entre P et Q."""
- **Finalité** : Mesure à quel point `Q` (2D) est différent de `P` (haute dimension).


    return np.sum(P * np.log(P / Q))
- **Effet** :  
  - Si `Q` est très différent de `P`, la divergence est grande.  
  - Si `Q ≈ P`, la divergence est proche de 0.
- **Finalité** : Utilisé pour suivre la qualité de l'embedding pendant l'optimisation.


In [ ]:
def fit(self, X):
        """Fit le modèle aux données X."""
        n_samples = X.shape[0]
        
        # Vérification des données
        if n_samples < 3 * self.perplexity:
            raise ValueError(f"Le nombre d'échantillons ({n_samples}) doit être au moins 3 * perplexity ({3*self.perplexity})")
        
        if self.verbose:
            print("Calcul des probabilités jointes P...")
        
        # Calcul des P en haute dimension
        P = self._compute_joint_probabilities(X, self.perplexity)
        P *= self.early_exaggeration
        
        # Initialisation aléatoire de Y
        Y = 1e-4 * np.random.randn(n_samples, self.n_components).astype(np.float32)
        
        # Variables pour l'optimisation
        previous_gradient = np.zeros_like(Y)
        gains = np.ones_like(Y)
        
        if self.verbose:
            print("Optimisation de l'embedding...")
        
        # Optimisation
        for i in range(self.n_iter):
            # Calcul des Q en basse dimension
            Q = self._compute_low_dimensional_probabilities(Y)
            
            # Calcul du gradient
            gradient = self._compute_gradient(P, Q, Y)
            grad_norm = np.linalg.norm(gradient)
            
            # Mise à jour avec momentum
            gains = (gains + 0.2) * ((gradient > 0) != (previous_gradient > 0)) + \
                    (gains * 0.8) * ((gradient > 0) == (previous_gradient > 0))
            gains = np.clip(gains, 0.01, np.inf)
            
            previous_gradient = gradient.copy()
            Y -= self.learning_rate * (gains * gradient)
            
            # Centrage des données
            Y = Y - np.mean(Y, axis=0)
            
            # Calcul de la divergence KL
            kl_div = self._compute_kl_divergence(P, Q)
            
            # Réduction de l'exagération après 100 itérations
            if i == 100:
                P /= self.early_exaggeration
            
            # Affichage des informations
            if self.verbose >= 1 and i % 100 == 0:
                print(f"Iteration {i}: KL divergence = {kl_div:.4f}, Gradient norm = {grad_norm:.4f}")
                
                if grad_norm < self.min_grad_norm:
                    if self.verbose:
                        print(f"Arrêt prématuré à l'itération {i}: norme du gradient trop faible")
                    break
        
        # Sauvegarde des résultats
        self.embedding_ = Y
        self.kl_divergence_ = kl_div
        self.n_iter_ = i + 1
        
        return self
    

def fit(self, X):
    """Fit le modèle aux données X."""
- **Finalité** : Entraîne le modèle t-SNE sur les données `X`.


    n_samples = X.shape[0]
- **Effet** : Stocke le nombre de points.


    if n_samples < 3 * self.perplexity:
        raise ValueError("...")
- **Effet** : Vérifie qu'il y a assez de points pour la perplexité choisie.
- **Finalité** : Évite des calculs instables (trop peu de voisins).


    P = self._compute_joint_probabilities(X, self.perplexity)
- **Effet** : Calcule les similarités `P` en haute dimension.
- **Finalité** : Ces similarités doivent être préservées en 2D/3D.


    P *= self.early_exaggeration
- **Effet** : Exagère les similarités au début (×12 par défaut).
- **Finalité** : Aide à séparer les clusters tôt dans l'optimisation.


    Y = 1e-4 * np.random.randn(n_samples, self.n_components)
- **Effet** : Initialise aléatoirement les positions en 2D/3D.
- **Finalité** : Point de départ pour l'optimisation.


    previous_gradient = np.zeros_like(Y)
    gains = np.ones_like(Y)
- **Effet** : Initialise les variables pour le momentum.
- **Finalité** : Accélère la convergence en évitant les oscillations.


    for i in range(self.n_iter):
        Q = self._compute_low_dimensional_probabilities(Y)
- **Effet** : Calcule `Q` (similarités en 2D/3D) à chaque itération.


        gradient = self._compute_gradient(P, Q, Y)
- **Effet** : Calcule comment déplacer les points pour rapprocher `Q` de `P`.


        gains = (gains + 0.2) * ((gradient > 0) != (previous_gradient > 0)) + \
                (gains * 0.8) * ((gradient > 0) == (previous_gradient > 0))
- **Effet** : Ajuste dynamiquement les gains pour accélérer la descente.
- **Finalité** : Si le gradient change de direction, augmente le gain. Sinon, le diminue.


        Y -= self.learning_rate * (gains * gradient)
- **Effet** : Met à jour les positions en suivant le gradient.
- **Finalité** : Déplace les points pour minimiser la divergence KL.


        Y = Y - np.mean(Y, axis=0)
- **Effet** : Centre les données pour éviter la dérive.
- **Finalité** : Garde l'embedding stable numériquement.


        if i == 100:
            P /= self.early_exaggeration
- **Effet** : Après 100 itérations, arrête l'exagération.
- **Finalité** : Permet un affinement plus précis des positions.


        if grad_norm < self.min_grad_norm:
            break
- **Effet** : Arrête l'optimisation si le gradient est trop petit.
- **Finalité** : Évite des itérations inutiles une fois convergé.


    self.embedding_ = Y
- **Finalité** : Stocke le résultat final (coordonnées 2D/3D).



In [ ]:
   def fit_transform(self, X):
        """Fit le modèle aux données et retourne l'embedding."""
        self.fit(X)
        return self.embedding_


def fit_transform(self, X):
    """Fit le modèle et retourne l'embedding."""
- **Finalité** : Combine `fit(X)` et retourne directement l'embedding.


    self.fit(X)
    return self.embedding_
- **Effet** : Entraîne le modèle et renvoie les coordonnées 2D/3D.



In [ ]:

def generate_test_data(n_samples=300, case='blobs', random_state=None):
    """Génère des données de test."""
    if random_state:
        np.random.seed(random_state)
    
    if case == 'blobs':
        # Données groupées en clusters
        centers = np.array([[1, 1, 1], [-1, -1, 1], [1, -1, -1], [-1, 1, -1]])
        X = np.vstack([center + np.random.randn(n_samples//4, 3)*0.3 for center in centers])
        y = np.repeat(np.arange(4), n_samples//4)
    elif case == 'swiss_roll':
        # Données en forme de rouleau suisse
        t = 1.5 * np.pi * (1 + 2 * np.random.rand(n_samples))
        X = np.vstack([t * np.cos(t), 10 * np.random.rand(n_samples), t * np.sin(t)]).T
        y = (t // np.pi).astype(int)
    else:
        # Données linéairement séparables
        X = np.random.randn(n_samples, 3)
        X[:n_samples//2] += 1
        X[n_samples//2:] -= 1
        y = np.zeros(n_samples)
        y[n_samples//2:] = 1
    
    # Normalisation
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X, y

Cette fonction génère des jeux de données synthétiques en 3D pour tester des algorithmes comme t-SNE. Voici ce que fait chaque partie du code :

## Initialisation et contrôle aléatoire
```python
def generate_test_data(n_samples=300, case='blobs', random_state=None):
    """Génère des données de test."""
    if random_state:
        np.random.seed(random_state)
```
- **Fonction** : Crée des données de test avec 3 options différentes
- **Paramètres** :
  - `n_samples` : nombre total de points (300 par défaut)
  - `case` : type de données ('blobs', 'swiss_roll' ou autre)
  - `random_state` : pour reproductibilité des résultats

## 1. Cas 'blobs' - Données groupées en clusters
```python
if case == 'blobs':
    centers = np.array([[1, 1, 1], [-1, -1, 1], [1, -1, -1], [-1, 1, -1]])
    X = np.vstack([center + np.random.randn(n_samples//4, 3)*0.3 for center in centers])
    y = np.repeat(np.arange(4), n_samples//4)
```
- **Structure** : 4 clusters gaussiens centrés autour de points en 3D
- **Génération** :
  - Crée 4 centres dans l'espace 3D
  - Pour chaque centre, génère `n_samples//4` points avec une distribution normale (bruit de 0.3)
- **Labels** : Chaque cluster a un label différent (0 à 3)

## 2. Cas 'swiss_roll' - Rouleau suisse
```python
elif case == 'swiss_roll':
    t = 1.5 * np.pi * (1 + 2 * np.random.rand(n_samples))
    X = np.vstack([t * np.cos(t), 10 * np.random.rand(n_samples), t * np.sin(t)]).T
    y = (t // np.pi).astype(int)
```
- **Structure** : Une spirale 3D (comme un rouleau de papier suisse)
- **Génération** :
  - `t` : Paramètre qui suit la longueur de la spirale
  - Coordonnées x et z : fonction cos/sin de t pour créer la spirale
  - Coordonnée y : aléatoire pour "étaler" le rouleau
- **Labels** : Basés sur la position angulaire (t // π)

## 3. Cas par défaut - Données linéairement séparables
```python
else:
    X = np.random.randn(n_samples, 3)
    X[:n_samples//2] += 1
    X[n_samples//2:] -= 1
    y = np.zeros(n_samples)
    y[n_samples//2:] = 1
```
- **Structure** : Deux groupes séparables linéairement
- **Génération** :
  - Tous les points suivent d'abord une distribution normale
  - On décale la 1ère moitié vers +1
  - On décale la 2nde moitié vers -1
- **Labels** : 0 pour le premier groupe, 1 pour le second

## Normalisation finale
```python
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
return X, y
```
- **Standardisation** :
  - Centre les données (moyenne = 0)
  - Met à l'échelle (écart-type = 1)
- **Retour** :
  - `X` : matrice (n_samples × 3) des features
  - `y` : vecteur (n_samples) des labels/clusters

## Exemples d'utilisation
```python
# 1. Génération de clusters
X, y = generate_test_data(case='blobs') 

# 2. Génération d'une spirale 3D
X, y = generate_test_data(n_samples=500, case='swiss_roll')

# 3. Données reproductibles
X, y = generate_test_data(random_state=42)
```

Cette fonction est particulièrement utile pour :
- Tester des algorithmes de réduction de dimension (t-SNE, PCA)
- Visualiser comment différents types de données se projettent en 2D
- Comparer des méthodes de clustering

In [ ]:
def plot_results(X, y, title, ax=None):
    """Visualise les résultats en 2D ou 3D."""
    if ax is None:
        fig = plt.figure()
        if X.shape[1] == 3:
            ax = fig.add_subplot(111, projection='3d')
        else:
            ax = fig.add_subplot(111)
    
    if X.shape[1] == 2:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', alpha=0.7)
    else:
        ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap='viridis', alpha=0.7)
    ax.set_title(title)
    ax.grid(True)

Cette fonction permet de visualiser des données en 2D ou 3D avec un codage couleur selon les labels. Voici son fonctionnement :

## Initialisation du graphique
```python
def plot_results(X, y, title, ax=None):
    """Visualise les résultats en 2D ou 3D."""
    if ax is None:
        fig = plt.figure()
        if X.shape[1] == 3:
            ax = fig.add_subplot(111, projection='3d')
        else:
            ax = fig.add_subplot(111)
```
- **Paramètres** :
  - `X` : matrice des données (2D ou 3D)
  - `y` : vecteur des labels/clusters
  - `title` : titre du graphique
  - `ax` : axe matplotlib existant (optionnel)

- **Fonctionnement** :
  - Si aucun axe n'est fourni (`ax=None`), crée une nouvelle figure
  - Détecte automatiquement si les données sont en 2D ou 3D (`X.shape[1]`)
  - Pour les données 3D, initialise une projection 3D

## Affichage des points
```python
    if X.shape[1] == 2:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', alpha=0.7)
    else:
        ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y, cmap='viridis', alpha=0.7)
```
- **2D** : 
  - Affiche les deux premières colonnes de X comme coordonnées x et y
  - Utilise `y` pour le codage couleur
  - `cmap='viridis'` : palette de couleurs
  - `alpha=0.7` : transparence pour mieux voir les superpositions

- **3D** :
  - Affiche les trois colonnes comme coordonnées x, y et z
  - Même principe de coloration que pour le 2D

## Finalisation du graphique
```python
    ax.set_title(title)
    ax.grid(True)
```
- Ajoute un titre au graphique
- Active une grille pour mieux visualiser les positions

## Exemples d'utilisation
```python
# Données 2D
X_2d = np.random.rand(100, 2)
y = np.random.randint(0, 3, 100)
plot_results(X_2d, y, "Données 2D aléatoires")

# Données 3D
X_3d = np.random.rand(100, 3)
plot_results(X_3d, y, "Données 3D aléatoires", ax=ax)  # Sur un axe existant
```

## Fonctionnalités clés
1. **Adaptation automatique** : gère aussi bien le 2D que le 3D
2. **Visualisation claire** : 
   - Couleurs par cluster/classe
   - Transparence pour voir les densités
   - Grille de référence
3. **Flexibilité** : peut s'intégrer dans une figure existante

Cette fonction est particulièrement utile pour :
- Visualiser les résultats de t-SNE/PCA
- Vérifier la qualité des clusters
- Comparer différentes projections de données

In [ ]:
def main():
    # Génération des données
    X, y = generate_test_data(300, case='blobs', random_state=42)
    
    # Visualisation des données originales
    fig = plt.figure(figsize=(15, 5))
    
    ax1 = fig.add_subplot(131)
    plot_results(X[:, :2], y, "Projection 2D originale", ax1)
    
    ax2 = fig.add_subplot(132, projection='3d')
    plot_results(X, y, "Données originales 3D", ax2)
    
    # Application de t-SNE
    tsne = TSNE(n_components=2, perplexity=30, 
                learning_rate=200, n_iter=1000,
                random_state=42, verbose=1)
    
    start_time = time.time()
    X_tsne = tsne.fit_transform(X)
    duration = time.time() - start_time
    
    # Visualisation des résultats
    ax3 = fig.add_subplot(133)
    plot_results(X_tsne, y, f"Donnée après t-SNE 2D (temps: {duration:.2f}s)", ax3)
    plt.tight_layout()
    plt.show()
    
    # Affichage des informations
    print("\nRésultats t-SNE:")
    print(f"Divergence KL finale: {tsne.kl_divergence_:.4f}")
    print(f"Itérations effectuées: {tsne.n_iter_}")

if __name__ == "__main__":
    main()

# Explication détaillée de la fonction `main()`

Cette fonction principale démontre un workflow complet de visualisation de données avec t-SNE. Voici son fonctionnement étape par étape :

## 1. Génération des données de test
```python
X, y = generate_test_data(300, case='blobs', random_state=42)
```
- **Action**: Crée un jeu de données synthétique
- **Paramètres**:
  - 300 points (`n_samples=300`)
  - Type 'blobs' (4 clusters gaussiens en 3D)
  - `random_state=42` pour la reproductibilité
- **Retourne**:
  - `X`: matrice 300x3 des features
  - `y`: vecteur des labels (0 à 3)

## 2. Configuration de la figure
```python
fig = plt.figure(figsize=(15, 5))
```
- Crée une figure large de 15 pouces par 5 pouces
- Permettra d'afficher 3 graphiques côte à côte

## 3. Visualisation des données originales
### Projection 2D
```python
ax1 = fig.add_subplot(131)
plot_results(X[:, :2], y, "Projection 2D originale", ax1)
```
- **131**: 1 ligne, 3 colonnes, 1ère position
- Affiche seulement les 2 premières dimensions
- Montre la perte d'information en 2D brute

### Vue 3D complète
```python
ax2 = fig.add_subplot(132, projection='3d')
plot_results(X, y, "Données originales 3D", ax2)
```
- **132**: 1 ligne, 3 colonnes, 2ème position
- `projection='3d'` pour l'affichage 3D
- Montre la structure complète des données

## 4. Application de t-SNE
```python
tsne = TSNE(n_components=2, perplexity=30, 
            learning_rate=200, n_iter=1000,
            random_state=42, verbose=1)
```
- **Configuration t-SNE**:
  - Réduction en 2D (`n_components=2`)
  - Perplexité moyenne (`perplexity=30`)
  - Taux d'apprentissage élevé (`learning_rate=200`)
  - 1000 itérations maximum
  - `verbose=1` pour afficher la progression

### Calcul et chronométrage
```python
start_time = time.time()
X_tsne = tsne.fit_transform(X)
duration = time.time() - start_time
```
- Mesure le temps d'exécution de t-SNE
- `fit_transform()` applique l'algorithme et retourne la projection 2D

## 5. Visualisation des résultats t-SNE
```python
ax3 = fig.add_subplot(133)
plot_results(X_tsne, y, f"Donnée après t-SNE 2D (temps: {duration:.2f}s)", ax3)
plt.tight_layout()
plt.show()
```
- **133**: 1 ligne, 3 colonnes, 3ème position
- Affiche la projection t-SNE avec:
  - Couleurs par cluster original
  - Temps d'exécution dans le titre
- `tight_layout()` améliore l'espacement
- `show()` affiche la figure

## 6. Affichage des métriques
```python
print("\nRésultats t-SNE:")
print(f"Divergence KL finale: {tsne.kl_divergence_:.4f}")
print(f"Itérations effectuées: {tsne.n_iter_}")
```
- Affiche:
  - La divergence KL (qualité de la projection)
  - Le nombre réel d'itérations effectuées

## 7. Exécution conditionnelle
```python
if __name__ == "__main__":
    main()
```
- Garantit que le code ne s'exécute que si le script est lancé directement (pas en import)

## Flux complet
1. Génère des données 3D avec 4 clusters
2. Montre:
   - Une projection 2D naïve (perte d'information)
   - La vue 3D originale (structure complète)
   - La projection t-SNE 2D (préservation des clusters)
3. Donne des métriques quantitatives

## Résultat attendu
- Visualisation comparative montrant comment t-SNE préserve mieux la structure des clusters qu'une simple projection 2D
- Affichage des performances de l'algorithme
- Démonstration complète du workflow t-SNE